##### Fèlix Fernández Peñafiel, 1523257

# Historic Battles Dataset Creation

First of all, we will start with the dataset battles.csv, importing it:

In [1]:
import pandas as pd

df_battles = pd.read_csv('data/battles.csv')
print(df_battles.shape)

(660, 49)


In [8]:
print(df_battles.at[df_battles[df_battles["isqno"] == 73].index[0], "wina"])


1.0


In [4]:
df_battles.dtypes

isqno              int64
war               object
name              object
locn              object
campgn            object
postype            int64
post1             object
post2             object
front            float64
depth            float64
time             float64
aeroa            float64
surpa            float64
cea              float64
leada            float64
trnga            float64
morala           float64
logsa            float64
momnta           float64
intela           float64
techa            float64
inita            float64
wina             float64
kmda             float64
crit               int64
quala            float64
resa             float64
mobila           float64
aira             float64
fprepa           float64
wxa              float64
terra            float64
leadaa           float64
plana            float64
surpaa           float64
mana             float64
logsaa           float64
fortsa           float64
deepa            float64
is_hero            int64


We start by filtering the columns of Battles.csv. The columns to eliminate are the ones that have obviously no relation to the outcome of a battle, like the name of the war, links to webs with additional information, atc.

In [5]:
battle_features_to_elim = ["parent", "war_initiator", "cow_warname", 
                           "cow_warno", "dbpedia", "war4_theater", "war2", "war3", "war4", "is_hero",
                           "war", "name", "locn", "campgn"]

battle_columns_to_keep = [col for col in df_battles.columns if col not in battle_features_to_elim]

df_battles = df_battles[battle_columns_to_keep]

We repeat the process with the *CSV* *Battle_Durations*, *Belligerents*, *Commanders*, *Front_Widths*, *Terrain* and *Weather*.

In [6]:
df_battle_durations = pd.read_csv('data/battle_durations.csv')
df_belligerents = pd.read_csv('data/belligerents.csv')
df_commanders = pd.read_csv('data/commanders.csv', encoding="unicode_escape")
df_front_widths = pd.read_csv('data/front_widths.csv')
df_terrain = pd.read_csv('data/terrain.csv')
df_weather = pd.read_csv('data/weather.csv')

In [7]:
battle_durations_features_to_elim = ["datetime_min", "datetime_max", "duration2", "datetime"]
belligerents_features_to_elim = ["co", "nam", "strpl", "strmi", "caspl", "casmi", "code", "actors"]
commanders_features_to_elim = ["actors", "uri"]
front_widths_features_to_elim = ["front_number", "time_min", "time_max"]
terrain_features_to_elim = ["terrano"]
weather_features_to_elim = ["wxno"]

In [8]:
battle_durations_columns_to_keep = [col for col in df_battle_durations.columns if col not in battle_durations_features_to_elim]
belligerents_columns_to_keep = [col for col in df_belligerents.columns if col not in belligerents_features_to_elim]
commanders_columns_to_keep = [col for col in df_commanders.columns if col not in commanders_features_to_elim]
front_widths_columns_to_keep = [col for col in df_front_widths.columns if col not in front_widths_features_to_elim]
terrain_columns_to_keep = [col for col in df_terrain.columns if col not in terrain_features_to_elim]
weather_columns_to_keep = [col for col in df_weather.columns if col not in weather_features_to_elim]


In [9]:
df_battle_durations = df_battle_durations[battle_durations_columns_to_keep]
df_belligerents = df_belligerents[belligerents_columns_to_keep]
df_commanders = df_commanders[commanders_columns_to_keep]
df_front_widths = df_front_widths[front_widths_columns_to_keep]
df_terrain = df_terrain[terrain_columns_to_keep]
df_weather = df_weather[weather_columns_to_keep]

Now, we need to start building the final *CSV*, combining all the others using the battle ID's. We start this with the most simple merging, using datasets we don't have manipulate its data for them to fit correctly.

In [10]:
df_combined = pd.merge(df_battles, df_battle_durations, on='isqno', how='outer') # how=inner makes it so
                                                                        # only rows with the same isqno in both
                                                                        # datasets are included in the new one

# Front_Widths contains multiple rows of the same battle, detailing what are the front widths at different times.
# We will ony look at the front widths at the start of the battle
df_front_widths = df_front_widths.drop_duplicates(subset="isqno", keep="first")
df_combined = pd.merge(df_combined, df_front_widths, on="isqno", how="outer")

df_combined = pd.merge(df_combined, df_terrain, on="isqno", how="outer")

df_combined = pd.merge(df_combined, df_weather, on="isqno", how="outer")

df_combined

,isqno,postype,post1,post2,front,depth,time,aeroa,surpa,cea,...,wofa,wofd,terra1,terra2,terra3,wx1,wx2,wx3,wx4,wx5
0,1,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,0.90,0.9,R,B,D,D,S,T,S,T
1,2,1,HD,PD,1.0,0.0,0.0,0.0,0.0,0.0,...,1.50,1.5,R,M,NaN,D,S,T,W,T
2,3,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,3.70,2.8,R,M,NaN,D,S,H,$,T
3,4,0,PD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,...,NaN,NaN,R,M,NaN,D,S,T,$,T
4,5,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,R,M,NaN,D,S,T,S,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,656,9,PD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,5.00,NaN,R,B,NaN,D,NaN,H,S,D
974,657,9,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,96.00,NaN,R,M,NaN,W,L,T,$,T
975,658,9,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,1.80,NaN,R,M,NaN,W,L,T,$,T
976,659,9,HD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,5.00,NaN,R,M,NaN,D,NaN,T,$,T


Now, the *Belligerents* *CSV*. This one is trickier, since there are two rows for each battle, one for the attacking force, and another for the defender. We want to merge each pair of rows into one, to be able to then combine it with the other datasets.

We will start separating attackers from defenders, so that we can rename the columns. We know a row is an attacker or defender depending on the attribute *attacker*, which is 0 if it's a defender and 1 if it's an attacker.

In [11]:
df_attackers = df_belligerents[df_belligerents["attacker"] == 1]
df_defenders = df_belligerents[df_belligerents["attacker"] == 0]

df_attackers = df_attackers[[col for col in df_attackers.columns if col not in ["attacker"]]]
df_defenders = df_defenders[[col for col in df_defenders.columns if col not in ["attacker"]]]

Now, we have to rename the attributes of the attacker and defender, adding a prefix so that we know which is which.

In [12]:
df_attackers = df_attackers.rename(columns={"str": "attacker_str", "intst": "attacker_instst", "rerp": "attacker_rerp", 
                             "cas": "attacker_cas", "finst": "attacker_finst", "cav": "attacker_cav", 
                             "tank": "attacker_tank", "lt": "attacker_lt", "mbt": "attacker_mbt", 
                             "arty": "attacker_arty", "fly": "attacker_fly", "ctank": "attacker_ctank",
                             "carty": "attacker_carty", "pri1": "attacker_pri1", "pri2": "attacker_pri2",
                             "pri3": "attacker_pri3", "sec1": "attacker_sec1","sec2": "attacker_sec2",
                             "sec3": "attacker_sec3", "reso1": "attacker_reso1", "reso2": "attacker_reso2",
                             "reso3": "attacker_reso3", "ach": "attacker_ach"})

df_defenders = df_defenders.rename(columns={"str": "defender_str", "intst": "defender_instst", "rerp": "defender_rerp", 
                             "cas": "defender_cas", "finst": "defender_finst", "cav": "defender_cav", 
                             "tank": "defender_tank", "lt": "defender_lt", "mbt": "defender_mbt", 
                             "arty": "defender_arty", "fly": "defender_fly", "ctank": "defender_ctank",
                             "carty": "defender_carty", "pri1": "defender_pri1", "pri2": "defender_pri2",
                             "pri3": "defender_pri3", "sec1": "defender_sec1","sec2": "defender_sec2",
                             "sec3": "defender_sec3", "reso1": "defender_reso1", "reso2": "defender_reso2",
                             "reso3": "defender_reso3", "ach": "defender_ach"})

Now, we merge the attackers and defenders dataset into one, belligerents:

In [13]:
attackers_duplicate_check = df_attackers["isqno"].value_counts().max()
defenders_duplicate_check = df_defenders["isqno"].value_counts().max()
print(f"Max duplicates in attackers: {attackers_duplicate_check}")
print(f"Max duplicates in defenders: {defenders_duplicate_check}")


Max duplicates in attackers: 1
Max duplicates in defenders: 1


In [14]:
df_belligerents = pd.merge(df_attackers, df_defenders, on="isqno", how="outer")
df_belligerents

,isqno,attacker_str,attacker_instst,attacker_rerp,attacker_cas,attacker_finst,attacker_cav,attacker_tank,attacker_lt,attacker_mbt,...,defender_pri1,defender_pri2,defender_pri3,defender_sec1,defender_sec2,defender_sec3,defender_reso1,defender_reso2,defender_reso3,defender_ach
0,1,11500.0,11500.0,0.0,4000.0,7500.0,1500.0,0.0,0.0,0.0,...,DO,FF,NaN,NaN,NaN,0,NaN,NaN,NaN,8
1,2,40000.0,40000.0,0.0,400.0,39600.0,7500.0,0.0,0.0,0.0,...,DO,FF,NaN,NaN,NaN,0,RR,WL,NaN,2
2,3,20000.0,20000.0,0.0,5000.0,15000.0,4000.0,0.0,0.0,0.0,...,DD,NaN,NaN,NaN,NaN,0,WL,NaN,NaN,3
3,4,8500.0,8500.0,0.0,3000.0,5500.0,2500.0,0.0,0.0,0.0,...,DO,DE,NaN,NaN,NaN,0,BB,PS,NaN,9
4,5,17000.0,17000.0,0.0,2000.0,15000.0,7000.0,0.0,0.0,0.0,...,DO,NaN,NaN,NaN,NaN,0,WL,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
655,656,4500.0,NaN,NaN,150.0,NaN,0.0,57.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0
656,657,NaN,NaN,NaN,NaN,NaN,0.0,2439.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,WD,NaN,NaN,0
657,658,17000.0,NaN,NaN,NaN,NaN,0.0,218.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,WD,NaN,NaN,0
658,659,11821.0,NaN,NaN,NaN,NaN,0.0,88.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,PS,NaN,NaN,0


And now we can merge belligerents with the combined dataset:

In [15]:
df_combined = pd.merge(df_combined, df_belligerents, on="isqno", how="outer")
df_combined

,isqno,postype,post1,post2,front,depth,time,aeroa,surpa,cea,...,defender_pri1,defender_pri2,defender_pri3,defender_sec1,defender_sec2,defender_sec3,defender_reso1,defender_reso2,defender_reso3,defender_ach
0,1,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,DO,FF,NaN,NaN,NaN,0,NaN,NaN,NaN,8
1,2,1,HD,PD,1.0,0.0,0.0,0.0,0.0,0.0,...,DO,FF,NaN,NaN,NaN,0,RR,WL,NaN,2
2,3,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,DD,NaN,NaN,NaN,NaN,0,WL,NaN,NaN,3
3,4,0,PD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,...,DO,DE,NaN,NaN,NaN,0,BB,PS,NaN,9
4,5,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,DO,NaN,NaN,NaN,NaN,0,WL,NaN,NaN,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,656,9,PD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,0
974,657,9,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,WD,NaN,NaN,0
975,658,9,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,WD,NaN,NaN,0
976,659,9,HD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0,PS,NaN,NaN,0


Now, the only dataset left to merge is *Commanders*. This one is also tricky, since, as *Belligerents*, this also has multiple rows pertaining to the same battle. On top of that, each battle may have more than one attacking or defending commander, normally depending on the number of nations involved in the struggle.

First of all, we must look at the maximum number of attacking and defending commanders.

Mirem el nombre màxim de defensors i atacants. De defensors son 3, atacants 2, així que haurem de fer 3 atributs pels defensors i 2 pels atacants.

In [16]:
att_comms = df_commanders[df_commanders["attacker"]==1]
def_comms = df_commanders[df_commanders["attacker"]==0]

In [17]:
isqno_counts_att = att_comms['isqno'].value_counts()
isqno_counts_att

isqno
52     2
165    2
69     2
53     2
230    2
      ..
231    1
232    1
233    1
234    1
660    1
Name: count, Length: 660, dtype: int64

In [18]:
isqno_counts_def = def_comms['isqno'].value_counts()
isqno_counts_def

isqno
186    3
50     3
136    2
18     2
112    2
      ..
232    1
233    1
234    1
235    1
660    1
Name: count, Length: 660, dtype: int64

We see that the maximum number of attacking commanders in the same battle is 2. Therefore, we will need to create two attributes, *att_comm1* and *att_comm2*. The maximum number of defensive commanders is 3, so we will need three different attributes.

We need to divide the commanders into attacking and defending generals:

In [19]:
att_comms = df_commanders[df_commanders["attacker"]==1]
def_comms = df_commanders[df_commanders["attacker"]==0]

In [20]:
att_comms

,isqno,attacker,commanders
1,1,1,ALBERT OF AUSTRIA
3,2,1,BUCQUOI
5,3,1,TILLY
6,3,1,CORDOVA
8,4,1,MANSFIELD
...,...,...,...
1349,656,1,NaN
1351,657,1,NaN
1353,658,1,NaN
1355,659,1,NaN


In [21]:
attacking_comm = att_comms.groupby('isqno')['commanders'].apply(lambda x: pd.Series(x.values).rename(lambda i: f'att_comm{i + 1}')).reset_index()
attacking_comm

,isqno,level_1,commanders
0,1,att_comm1,ALBERT OF AUSTRIA
1,2,att_comm1,BUCQUOI
2,3,att_comm1,TILLY
3,3,att_comm2,CORDOVA
4,4,att_comm1,MANSFIELD
...,...,...,...
671,656,att_comm1,NaN
672,657,att_comm1,NaN
673,658,att_comm1,NaN
674,659,att_comm1,NaN


In [22]:
defending_comm = def_comms.groupby('isqno')['commanders'].apply(lambda x: pd.Series(x.values).rename(lambda i: f'def_comm{i + 1}')).reset_index()
defending_comm

,isqno,level_1,commanders
0,1,def_comm1,MAURICE OF NASSAU
1,2,def_comm1,CHRISTIAN OF ANHALT-BERNBERG
2,3,def_comm1,BADEN-DURLACH
3,4,def_comm1,WALLENSTEIN
4,5,def_comm1,CHRISTIAN IV
...,...,...,...
677,656,def_comm1,NaN
678,657,def_comm1,NaN
679,658,def_comm1,NaN
680,659,def_comm1,NaN


In [23]:
attacking_comms = attacking_comm.pivot(index="isqno", columns="level_1", values="commanders")
attacking_comms.reset_index(inplace=True)
attacking_comms

level_1,isqno,att_comm1,att_comm2
0,1,ALBERT OF AUSTRIA,NaN
1,2,BUCQUOI,NaN
2,3,TILLY,CORDOVA
3,4,MANSFIELD,NaN
4,5,TILLY,NaN
...,...,...,...
655,656,NaN,NaN
656,657,NaN,NaN
657,658,NaN,NaN
658,659,NaN,NaN


In [24]:
defending_comms = defending_comm.pivot(index="isqno", columns="level_1", values="commanders")
defending_comms.reset_index(inplace=True)
defending_comms

level_1,isqno,def_comm1,def_comm2,def_comm3
0,1,MAURICE OF NASSAU,NaN,NaN
1,2,CHRISTIAN OF ANHALT-BERNBERG,NaN,NaN
2,3,BADEN-DURLACH,NaN,NaN
3,4,WALLENSTEIN,NaN,NaN
4,5,CHRISTIAN IV,NaN,NaN
...,...,...,...,...
655,656,NaN,NaN,NaN
656,657,NaN,NaN,NaN
657,658,NaN,NaN,NaN
658,659,NaN,NaN,NaN


In [ ]:
attacking_comms.columns.name = None 
attacking_comms.reset_index(inplace=True)  

attacking_comms.drop(columns=["level_1"], inplace=True, errors='ignore')
print(attacking_comms)

     index  isqno          att_comm1 att_comm2
0        0      1  ALBERT OF AUSTRIA       NaN
1        1      2            BUCQUOI       NaN
2        2      3              TILLY   CORDOVA
3        3      4          MANSFIELD       NaN
4        4      5              TILLY       NaN
..     ...    ...                ...       ...
655    655    656                NaN       NaN
656    656    657                NaN       NaN
657    657    658                NaN       NaN
658    658    659                NaN       NaN
659    659    660                NaN       NaN

[660 rows x 4 columns]


In [ ]:
defending_comms.columns.name = None
defending_comms.reset_index(inplace=True)

defending_comms.drop(columns=["level_1"], inplace=True, errors='ignore')
print(defending_comms)


     index  isqno                     def_comm1 def_comm2 def_comm3
0        0      1             MAURICE OF NASSAU       NaN       NaN
1        1      2  CHRISTIAN OF ANHALT-BERNBERG       NaN       NaN
2        2      3                 BADEN-DURLACH       NaN       NaN
3        3      4                   WALLENSTEIN       NaN       NaN
4        4      5                  CHRISTIAN IV       NaN       NaN
..     ...    ...                           ...       ...       ...
655    655    656                           NaN       NaN       NaN
656    656    657                           NaN       NaN       NaN
657    657    658                           NaN       NaN       NaN
658    658    659                           NaN       NaN       NaN
659    659    660                           NaN       NaN       NaN

[660 rows x 5 columns]


In [27]:
attacking_comms.drop(columns=["index"], inplace=True, errors='ignore')
print(attacking_comms)

     isqno          att_comm1 att_comm2
0        1  ALBERT OF AUSTRIA       NaN
1        2            BUCQUOI       NaN
2        3              TILLY   CORDOVA
3        4          MANSFIELD       NaN
4        5              TILLY       NaN
..     ...                ...       ...
655    656                NaN       NaN
656    657                NaN       NaN
657    658                NaN       NaN
658    659                NaN       NaN
659    660                NaN       NaN

[660 rows x 3 columns]


In [28]:
defending_comms.drop(columns=["index"], inplace=True, errors='ignore')
print(defending_comms)

     isqno                     def_comm1 def_comm2 def_comm3
0        1             MAURICE OF NASSAU       NaN       NaN
1        2  CHRISTIAN OF ANHALT-BERNBERG       NaN       NaN
2        3                 BADEN-DURLACH       NaN       NaN
3        4                   WALLENSTEIN       NaN       NaN
4        5                  CHRISTIAN IV       NaN       NaN
..     ...                           ...       ...       ...
655    656                           NaN       NaN       NaN
656    657                           NaN       NaN       NaN
657    658                           NaN       NaN       NaN
658    659                           NaN       NaN       NaN
659    660                           NaN       NaN       NaN

[660 rows x 4 columns]


Finally, we can combine the attacking and defending commanders.To check it's all correct, we look at the battle number 50, since it's the battle with the most defensive and attacking commanders (all in all, 5).

In [29]:
df_combined_comms = pd.merge(attacking_comms, defending_comms, on="isqno", how="outer")
print(df_combined_comms[df_combined_comms["isqno"]==50])

    isqno    att_comm1 att_comm2 def_comm1 def_comm2                def_comm3
49     50  MARLBOROUGH    EUGENE   TALLARD    MARSIN  MAX EMMANUEL OF BAVARIA


Now, we can merge the commanders dataframe into the combined dataframe from earlier, resulting into our final dataframe:

In [30]:
duplicates = df_combined[df_combined["isqno"].duplicated(keep=False)]
print(duplicates[duplicates["isqno"]==7])

Empty DataFrame
Columns: [isqno, postype, post1, post2, front, depth, time, aeroa, surpa, cea, leada, trnga, morala, logsa, momnta, intela, techa, inita, wina, kmda, crit, quala, resa, mobila, aira, fprepa, wxa, terra, leadaa, plana, surpaa, mana, logsaa, fortsa, deepa, duration1, wofa, wofd, terra1, terra2, terra3, wx1, wx2, wx3, wx4, wx5, attacker_str, attacker_instst, attacker_rerp, attacker_cas, attacker_finst, attacker_cav, attacker_tank, attacker_lt, attacker_mbt, attacker_arty, attacker_fly, attacker_ctank, attacker_carty, cfly_x, attacker_pri1, attacker_pri2, attacker_pri3, attacker_sec1, attacker_sec2, attacker_sec3, attacker_reso1, attacker_reso2, attacker_reso3, attacker_ach, defender_str, defender_instst, defender_rerp, defender_cas, defender_finst, defender_cav, defender_tank, defender_lt, defender_mbt, defender_arty, defender_fly, defender_ctank, defender_carty, cfly_y, defender_pri1, defender_pri2, defender_pri3, defender_sec1, defender_sec2, defender_sec3, defender_reso

In [31]:
df_historic_battles = pd.merge(df_combined, df_combined_comms, on="isqno", how="outer")
df_historic_battles = df_historic_battles.drop_duplicates(subset="isqno")
df_historic_battles

,isqno,postype,post1,post2,front,depth,time,aeroa,surpa,cea,...,defender_sec3,defender_reso1,defender_reso2,defender_reso3,defender_ach,att_comm1,att_comm2,def_comm1,def_comm2,def_comm3
0,1,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,NaN,...,0,NaN,NaN,NaN,8,ALBERT OF AUSTRIA,NaN,MAURICE OF NASSAU,NaN,NaN
1,2,1,HD,PD,1.0,0.0,0.0,0.0,0.0,0.0,...,0,RR,WL,NaN,2,BUCQUOI,NaN,CHRISTIAN OF ANHALT-BERNBERG,NaN,NaN
2,3,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0,WL,NaN,NaN,3,TILLY,CORDOVA,BADEN-DURLACH,NaN,NaN
3,4,0,PD,NaN,0.0,0.0,0.0,0.0,NaN,0.0,...,0,BB,PS,NaN,9,MANSFIELD,NaN,WALLENSTEIN,NaN,NaN
4,5,0,HD,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0,WL,NaN,NaN,3,TILLY,NaN,CHRISTIAN IV,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,656,9,PD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
974,657,9,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,0,WD,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
975,658,9,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,0,WD,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
976,659,9,HD,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,0,PS,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


Last of all, there are 219 battles with a NaN in *wina*. This will be introduced manually:

In [32]:
df_historic_battles.loc[df_historic_battles['isqno'].isin([1, 4, 6, 8, 10, 16, 22, 24, 25, 31, 39, 41, 45, 49, 
                                                         55, 56, 58, 63, 66, 67, 69, 74, 76, 77, 80, 83, 88, 
                                                         89, 92, 95, 96, 98, 101, 107, 109, 110, 115, 116, 117,
                                                         119, 122, 123, 124, 128, 129, 130, 131, 135, 137, 141,
                                                         144, 145, 147, 148, 149, 153, 155, 159, 166, 170, 171,
                                                         172, 174, 175, 176, 177, 178, 181, 183, 184, 186, 187, 
                                                         188, 189, 194, 196, 199, 202, 203, 205, 206, 207, 208, 
                                                         212, 213, 215, 229, 231, 234, 237, 238, 239, 245, 246,
                                                         255, 257, 258, 264, 269, 275, 276, 277, 278, 284, 285,
                                                         287, 289, 290, 291, 292, 293, 296, 298, 301, 303, 304, 
                                                         306, 310, 312, 317, 326, 327, 334, 338, 340, 343, 344,
                                                         345, 350, 351, 384, 386, 391, 392, 397, 398, 399, 402,
                                                         409, 413, 415, 417, 420, 424, 427, 430, 431, 433, 449,
                                                         451, 452, 459, 461, 467, 485, 489, 496, 525, 526, 527,
                                                         528, 531, 532, 540, 561, 564, 573, 574, 575, 579, 586,
                                                         588, 589, 592, 593, 596, 598, 599, 600, 603, 605, 606,
                                                         607, 608, 609, 611, 612, 613, 614, 615, 616, 619, 626,
                                                         635, 637, 639, 656, 659]), 'wina'] = 0
df_historic_battles.loc[df_historic_battles['isqno'].isin([111, 185, 191, 192, 209, 262, 274, 297, 299, 305,
                                                           319, 396, 405, 416, 447, 466, 469, 509, 529, 583,
                                                           640, 650]), 'wina'] = 1

For clarification, indecisive battles have been classified as victories for the defender.

Going through the dataset, I've noticed Napoleon Bonaparte is sometimes referred to Bonaparte, and others as Napoleon I. I've decided to unify these into Napoleon. There might be other instances of other commanders that are referenced with two different names throughout the dataset, but checking every single commander and comparing it with every single other is a task I do not have the time for. However, I expect that these instances are very low in number.

In [ ]:
commander_columns = ['att_comm1', 'att_comm2', 'def_comm1', 'def_comm2', 'def_comm3']

df_historic_battles[commander_columns] = df_historic_battles[commander_columns].replace(['NAPOLEON I', 'BONAPARTE'], 'NAPOLEON')

Finally, I wanted to add a "Missing" value to fill every space in the commander attributes that was NaN, and the attribute *wx2* has mistaken *Overcast* values with NaNs:

In [35]:
columns_to_impute = ['att_comm1', 'att_comm2', 'def_comm1', 'def_comm2', 'def_comm3']
df_historic_battles[columns_to_impute] = df_historic_battles[columns_to_impute].fillna('Missing')
df_historic_battles['wx2'] = df_historic_battles['wx2'].fillna('O')

For the sake of clarity, we'll form a *CSV* file of this dataset:

In [1]:
df_historic_battles.to_csv('History_Battles.csv', index=False)

NameError: name 'df_historic_battles' is not defined